## 2.  핫플지수 확정



In [1]:
import polars as pl
import numpy as np

print("✅ 1️⃣ Parquet 스캔 시작")
lazy_df = pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
print("   → LazyFrame 생성 완료")


✅ 1️⃣ Parquet 스캔 시작
   → LazyFrame 생성 완료


In [2]:
import polars as pl

# ———————— 풀 테이블 출력 설정 ————————
pl.Config.set_tbl_rows(100)         # 최대 100행까지 줄임표 없이 표시
pl.Config.set_tbl_cols(100)        # 최대 100열까지 표시
pl.Config.set_tbl_formatting("UTF8_FULL")  # 테두리 포함, 생략 없이 전체 보여주기


polars.config.Config

In [3]:

import polars as pl


# 1️⃣ filtered 정의: 2024년··주요 목적 필터 + 날짜 파싱
filtered = (
    lazy_df
      # ─ 날짜 범위: 2024-01-01 ~ 2024-12-31
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      
    #   # ─ 시간 범위: 18시(18)부터 23시(23)까지
    #   .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) 
    #         & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      
    #   # ─ 목적 필터: 관심 목적군만
    #   .filter(pl.col("MOVE_PURPOSE").is_in(
    #       ["쇼핑","관광","출근","귀가","병원","기타","등교"]
    #   ))


      # ─ ETL_YMD 문자열을 Date 타입으로 바꿔서
      .with_columns(
          pl.col("ETL_YMD")
            .str.strptime(pl.Date, format="%Y%m%d")
            .alias("ETL_DATE")
      )

#       # ─ 평일(월~금)만 남기기 (weekday 0~4)
#       .filter(pl.col("ETL_DATE").dt.weekday().is_in([0,1,2,3,4]))
      
)

# 이제 `filtered`에 “원하는 기간·시간·목적·요일” 이 모두 적용되어 있으니,
# 아래처럼 곧장 동별 연간 통계 계산에 쓰실 수 있습니다.



In [ ]:
# 1️⃣ STEP3 만들기

young_cols = ['MALE_20_CNT','MALE_30_CNT','FEML_20_CNT','FEML_30_CNT']
all_cols = young_cols + [ 
    'MALE_00_CNT','MALE_10_CNT','MALE_40_CNT','MALE_50_CNT','MALE_60_CNT','MALE_70_CNT',
    'FEML_00_CNT','FEML_10_CNT','FEML_40_CNT','FEML_50_CNT','FEML_60_CNT','FEML_70_CNT'
]

import polars as pl

# (1) 20·30대 합계와 전체 합계 Expr 정의
young_sum: pl.Expr = sum(
    (pl.col(c).cast(pl.Float64) for c in young_cols),
    pl.lit(0.0)
)         # “한 행(row)에서” 20·30대 합계

total_sum: pl.Expr = sum(
    (pl.col(c).cast(pl.Float64) for c in all_cols),
    pl.lit(0.0)
)         # “한 행(row)에서” 전체 방문자 합계


#  날짜·시간·목적 필터  
step1 = (
    lazy_df
      .filter((pl.col('ETL_YMD') >= '20240101') & (pl.col('ETL_YMD') <= '20241231'))
    #   .filter((pl.col('TIME_CD').cast(pl.Int64) >= 10) & (pl.col('TIME_CD').cast(pl.Int64) <= 23))
      .filter(pl.col('MOVE_PURPOSE').is_in(['쇼핑','관광','출근','귀가','병원','기타']))
)


step2 = (
    step1
      .with_columns(pl.col('ETL_YMD').str.strptime(pl.Date, format='%Y%m%d').alias('ETL_DATE'))
    #   .filter(pl.col('ETL_DATE').dt.weekday().is_in([5, 6]))
)

# 4-1️⃣ 먼저 YOUNG_VISITORS, TOTAL_VISITORS, PURPOSE_FLAG, YOUNG_RATIO 만 추가
step3 = step2.with_columns([
    young_sum.alias("YOUNG_VISITORS"),  #2030 합계
    total_sum.alias("TOTAL_VISITORS"),  #전체 합꼐

    (young_sum / (total_sum + 1e-6)).alias("YOUNG_RATIO"), #전체 중 2030 비율
])

print("\n▶ step3 샘플:")
print(step3
  .select(["YOUNG_RATIO", "FULL_NM","YOUNG_VISITORS","TOTAL_VISITORS", "ETL_DATE"])
  .limit(50)
  .collect()
)



▶ step3 샘플:
shape: (50, 5)
┌─────────────┬────────────────────────────────┬────────────────┬────────────────┬────────────┐
│ YOUNG_RATIO ┆ FULL_NM                        ┆ YOUNG_VISITORS ┆ TOTAL_VISITORS ┆ ETL_DATE   │
│ ---         ┆ ---                            ┆ ---            ┆ ---            ┆ ---        │
│ f64         ┆ str                            ┆ f64            ┆ f64            ┆ date       │
╞═════════════╪════════════════════════════════╪════════════════╪════════════════╪════════════╡
│ 0.224515    ┆ 서울특별시_중구_신당5동        ┆ 27.42          ┆ 122.13         ┆ 2024-10-01 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0.414714    ┆ 서울특별시_동작구_상도3동      ┆ 12.74          ┆ 30.72          ┆ 2024-10-01 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0.153586    ┆ 경기도_하남시_덕풍1동          ┆ 9.08           ┆ 59.12          ┆ 2024-10-01 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [7]:
import polars as pl

# ── (기존) step2, step3 까지 그대로 두세요 ──
# step2 = …
# step3 = step2.with_columns([... young_sum, total_sum, YOUNG_RATIO ...])

# ── 1) “20·30대 중 쇼핑&관광” 방문자 합계 Expr ──
shop_tour_young: pl.Expr = sum(
    (
      pl.when(pl.col("MOVE_PURPOSE").is_in(["쇼핑","관광"]))
        .then(pl.col(c).cast(pl.Float64))
        # .otherwise(0.0)
      for c in young_cols
    ),
    pl.lit(0.0)
).alias("ROW_SHOP_YOUNG")


# ── 2) step3 에 SHOP_YOUNG 추가 ──
step3_mod = step3.with_columns([shop_tour_young])

# 3️⃣ 샘플 출력 (한 행당 ROW_SHOP_YOUNG 값 보기)
print(step3_mod
  .select(["FULL_NM","ETL_DATE","YOUNG_VISITORS","ROW_SHOP_YOUNG"])
  .limit(10)
  .collect()
)


shape: (10, 4)
┌──────────────────────────────┬────────────┬────────────────┬────────────────┐
│ FULL_NM                      ┆ ETL_DATE   ┆ YOUNG_VISITORS ┆ ROW_SHOP_YOUNG │
│ ---                          ┆ ---        ┆ ---            ┆ ---            │
│ str                          ┆ date       ┆ f64            ┆ f64            │
╞══════════════════════════════╪════════════╪════════════════╪════════════════╡
│ 서울특별시_중구_신당5동      ┆ 2024-10-01 ┆ 27.42          ┆ null           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_동작구_상도3동    ┆ 2024-10-01 ┆ 12.74          ┆ null           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_하남시_덕풍1동        ┆ 2024-10-01 ┆ 9.08           ┆ null           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_양천구_목1동      ┆ 2024-10-01 ┆ 762.54         ┆ null           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌

In [9]:

# ── 3) 동별 연간 집계 ──
annual2 = (
    step3_mod
      .group_by("FULL_NM")
      .agg([
        pl.sum("ROW_SHOP_YOUNG").alias("ANNUAL_SHOP_YOUNG"),  # 분자
        pl.sum("YOUNG_VISITORS").alias("ANNUAL_YOUNG"),      # 분모 (20·30대 총)
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_TOTAL"),      # 로그 인자
      ])
      .with_columns([
        # √(쇼핑&관광) / √(20·30대)
        (
          pl.col("ANNUAL_SHOP_YOUNG").sqrt()
          / pl.col("ANNUAL_YOUNG").sqrt()
        ).alias("FRAC_SHOP_YOUNG"),

        # log(전체 + 1e6)
        ( (pl.col("ANNUAL_TOTAL") + 1e6).log() )
          .alias("LOG_TOTAL_PLUS1e6")
      ])
      .collect()  # 동 단위 수백 행만 로드
)
# # 동 5개만 샘플 출력
# print(annual2
#   .select(["FULL_NM","ANNUAL_SHOP_YOUNG","ANNUAL_YOUNG","ANNUAL_TOTAL"])
#   .limit(5)
#   .collect()
# )


In [10]:

# ── 4) HOTNESS2 계산 & 상위30 추출 ──
top30_h2 = (
    annual2
      .with_columns([
        # 최종 곱셈
        (
          pl.col("FRAC_SHOP_YOUNG")
          * pl.col("LOG_TOTAL_PLUS1e6")
        ).alias("HOTNESS2")
      ])
      .sort("HOTNESS2", descending=True)
      .limit(30)
)

print("▶ 동별 연간 HOTNESS2 TOP30:")
print(top30_h2.select(["FULL_NM","HOTNESS2"]))

# ── 5) 상위30 평균·중앙값 ──
mean_h2   = top30_h2["HOTNESS2"].mean()
median_h2 = top30_h2["HOTNESS2"].median()
print(f"\n▶ HOTNESS2 상위30 평균 : {mean_h2:.6f}")
print(f"▶ HOTNESS2 상위30 중앙값: {median_h2:.6f}")



▶ 동별 연간 HOTNESS2 TOP30:
shape: (30, 2)
┌────────────────────────────────┬──────────┐
│ FULL_NM                        ┆ HOTNESS2 │
│ ---                            ┆ ---      │
│ str                            ┆ f64      │
╞════════════════════════════════╪══════════╡
│ 인천광역시_미추홀구_관교동     ┆ 9.438812 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_송파구_잠실3동      ┆ 9.115935 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_강서구_방화2동      ┆ 8.676549 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_성남시_분당구_백현동    ┆ 8.573788 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_영등포구_영등포본동 ┆ 8.111522 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_이천시_호법면           ┆ 7.751826 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_수원시_권선구_서둔동    ┆ 7.751037 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_여주시_여흥동           ┆ 7.30921  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_서초구_반포4동      ┆ 7.25009  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 경기

In [11]:
# ── 6) 난향동 HOTNESS2 ──
nanhyang_h2 = (
    annual2
      .filter(pl.col("FULL_NM") == "서울특별시_관악구_난향동")
      .with_columns([
        (pl.col("FRAC_SHOP_YOUNG") * pl.col("LOG_TOTAL_PLUS1e6"))
          .alias("HOTNESS2")
      ])
      .select("HOTNESS2")
)
print("\n▶ 난향동 연간 HOTNESS2:")
print(nanhyang_h2)



▶ 난향동 연간 HOTNESS2:
shape: (1, 1)
┌──────────┐
│ HOTNESS2 │
│ ---      │
│ f64      │
╞══════════╡
│ 0.0      │
└──────────┘


In [ ]:
# 🔍 난향동(‘서울특별시_관악구_난향동’) 1년치 “쇼핑” & “관광” 목적 행 전부 보기
  # 아예 0이라 다른 최솟값으로 대치
  
import polars as pl

# 1️⃣ Parquet 스캔 & 2024년·목적 필터 적용
shop_tour_2024 = (
    pl.scan_parquet("combined_parquet/**/*.parquet", glob=True)
      # ─ 2024년 데이터만
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      # ─ 행정동 필터 (난향동)
      .filter(pl.col("FULL_NM") == "서울특별시_관악구_난향동")
      # ─ “쇼핑” 또는 “관광” 목적만
      .filter(pl.col("MOVE_PURPOSE").is_in(["쇼핑","관광"]))
      # ─ ETL_YMD → 날짜 변환 (원하시면)
      .with_columns(
          pl.col("ETL_YMD")
            .str.strptime(pl.Date, "%Y%m%d")
            .alias("ETL_DATE")
      )
      # ─ 실행
      .collect()
)

# 2️⃣ 결과 출력
print("▶ 난향동 2024년 쇼핑/관광 목적 행 전체:")
print(shop_tour_2024)


▶ 난향동 2024년 쇼핑/관광 목적 행 전체:
shape: (0, 27)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ D_A ┆ TIM ┆ MOV ┆ MAL ┆ MAL ┆ MAL ┆ MAL ┆ MAL ┆ MAL ┆ MAL ┆ MAL ┆ FEM ┆ FEM ┆ FEM ┆ FEM ┆ FEM ┆ FEM ┆ FEM ┆ FEM ┆ TOT ┆ ETL ┆ ADM ┆ SID ┆ SGG ┆ ADM ┆ FUL ┆ ETL │
│ DMD ┆ E_C ┆ E_P ┆ E_0 ┆ E_1 ┆ E_2 ┆ E_3 ┆ E_4 ┆ E_5 ┆ E_6 ┆ E_7 ┆ L_0 ┆ L_1 ┆ L_2 ┆ L_3 ┆ L_4 ┆ L_5 ┆ L_6 ┆ L_7 ┆ AL_ ┆ _YM ┆ I_C ┆ O_N ┆ _NM ┆ I_N ┆ L_N ┆ _DA │
│ ONG ┆ D   ┆ URP ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ 0_C ┆ CNT ┆ D   ┆ D   ┆ M   ┆ --- ┆ M   ┆ M   ┆ TE  │
│ _CD ┆ --- ┆ OSE ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ NT  ┆ --- ┆ --- ┆ --- ┆ --- ┆ str ┆ --- ┆ --- ┆ --- │
│ --- ┆ str ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ str ┆ str ┆ str ┆ str 

In [12]:
# ─── 1️⃣ annual2에 HOTNESS2 컬럼 붙이기 ─────────────────────────
annual2_with_h2 = annual2.with_columns([
    (
      pl.col("FRAC_SHOP_YOUNG")   # √(쇼핑&관광)
      * pl.col("LOG_TOTAL_PLUS1e6")  # × log(전체+1e6)
    ).alias("HOTNESS2")
])

# ─── 2️⃣ HOTNESS2가 0인 행 제외하고 최소값 뽑기 ────────────────
min_positive_h2 = (
    annual2_with_h2
      .filter(pl.col("HOTNESS2") > 0)              # 0 초과만
      .select(pl.col("HOTNESS2").min())            # 최소값
      .to_series()[0]                              # 스칼라로 추출
)

print(f"▶ HOTNESS2 > 0 중 최소값: {min_positive_h2:.6f}")


▶ HOTNESS2 > 0 중 최소값: 0.015250


In [ ]:
# ─── 1️⃣ 관광 목적만 필터 & TOTAL_CNT 캐스트 ─────────────────────────
tourism_df = (
    filtered
      .filter(pl.col("MOVE_PURPOSE") == "관광")
      .with_columns(
          pl.col("TOTAL_CNT")
            .cast(pl.Float64)
            .alias("TOTAL_CNT_F")
      )
)

# ─── 2️⃣ 동별 연간 관광 TOTAL_CNT 합산 ─────────────────────────────
tourism_by_dong = (
    tourism_df
      .group_by("FULL_NM")
      .agg(
          pl.sum("TOTAL_CNT_F").alias("ANNUAL_TOURISM_CNT")
      )
      .sort("ANNUAL_TOURISM_CNT", descending=True)
      .collect()
)

print("▶ 동별 연간 관광 TOTAL_CNT 합계:")
print(tourism_by_dong)
